In [ ]:
# Standard Libraries
import warnings

# Data Handling
import pandas as pd
import numpy as np

# Visualization
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

# Time Series Analysis
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_pacf
from prophet import Prophet
import timesfm
from sklearn.model_selection import TimeSeriesSplit

# Feature Engineering & Transformation
from sklearn.preprocessing import OrdinalEncoder, FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import RFE
from numpy import fft

# Metrics
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# Machine Learning Models
import xgboost as xgb
import lightgbm as lgb

# PyTorch and Deep Learning
import torch
import torch.nn as nn
import torch.optim as optim

# Suppress Warnings
warnings.filterwarnings("ignore")


In [ ]:
tfm = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="gpu",
          per_core_batch_size=32,
          horizon_len=128,
          num_layers=50,
          use_positional_embedding=False,
          context_len=2048,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-2.0-500m-pytorch"),
  )

### Load data

In [ ]:
df_omie_b = pd.read_csv('../../data/df_omie_blind.csv')
df_omie_l = pd.read_csv('../../data/df_omie_labelled.csv')
filtered_cat = pd.read_csv('../../data/filtered_categories.csv')
unit_list = pd.read_csv('../../data/unit_list.csv')

In [ ]:
data = df_omie_l.merge(unit_list, on='Codigo', how='left')
data = data.merge(filtered_cat, on='Codigo', how='left')
codes = filtered_cat['Codigo'].unique()
data = data[data['Codigo'].isin(codes)]

## Feature Engineering

In [ ]:
def time_features(df: pd.DataFrame):
    df['fechaHora'] = pd.to_datetime(df['fechaHora'])
    df['date'] = df['fechaHora'].dt.date
    df['hour'] = df['fechaHora'].dt.hour
    df['day_of_week'] = df['fechaHora'].dt.dayofweek  # Monday=0, Sunday=6
    df['month'] = df['fechaHora'].dt.month
    df['day_of_month'] = df['fechaHora'].dt.day
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    df.sort_values(['fechaHora', 'Codigo'], inplace=True)
    df['t'] = (df['fechaHora'] - df['fechaHora'].min()).dt.total_seconds() / 3600
    
    def sin_cos_features(df: pd.DataFrame, period, K, time_col='t'):
        df = df.sort_values(['Codigo', 'fechaHora'])
        for k in range(1, K + 1):
            df[f'sin_{period}_{k}'] = np.sin(2 * np.pi * k * df[time_col] / period)
            df[f'cos_{period}_{k}'] = np.cos(2 * np.pi * k * df[time_col] / period)
        return df
    
    df = sin_cos_features(df, period=24, K=3)
    
    return df    

def cyclical_features(df: pd.DataFrame):
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['dow_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['dow_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    return df
        
def interaction_features(df: pd.DataFrame):
    df['energia_hour_sin'] = df['lag_Energia'] * df['hour_sin']
    return df
    
def lags_features(df: pd.DataFrame):
    df.sort_values(['fechaHora'], inplace=True)
    df['lag_PrecEuro'] = df.groupby('Codigo')['PrecEuro'].shift(24*28)
    df['lag_Energia'] = df.groupby('Codigo')['Energia'].shift(24*28)
    df['lag_Energia'] = np.log(df['lag_Energia'] + 1)
    df['lag1_Energia'] = df.groupby('Codigo')['lag_Energia'].shift(1)
    df['lag24_Energia'] = df.groupby('Codigo')['lag_Energia'].shift(24)
    return df

In [ ]:
def feature_engineering(data: pd.DataFrame):
    data['Energia_stationary'] = data['Energia'].diff()
    data = time_features(data)
    data = cyclical_features(data)
    data = lags_features(data)
    data = interaction_features(data)
    data = data.sort_values(['fechaHora', 'Codigo'])
    return data

In [ ]:
data = feature_engineering(data)

## Feature Transformation

In [ ]:
class OrdinalEncodingTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        # Initialize the encoder with any desired options.
        self.encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
        self.cat_cols = None

    def fit(self, X: pd.DataFrame, y=None):
        # Identify categorical columns (dtype object) in the training data.
        self.cat_cols = X.select_dtypes(include=['object']).columns
        # Fit the encoder on the categorical columns.
        self.encoder.fit(X[self.cat_cols])
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        X_transformed = X.copy()
        if self.cat_cols is not None:
            X_transformed[self.cat_cols] = self.encoder.transform(X_transformed[self.cat_cols])
        return X_transformed

In [ ]:
def agg_features(df: pd.DataFrame):
    df['cum_energy'] = df.groupby(['Codigo', 'date'])['lag_Energia'].cumsum()
    return df

def rolling_mean_features(df: pd.DataFrame):
    features = ['lag_Energia', 'lag_PrecEuro']
    groups = ['Codigo', 'Categoria']
    times = [12, 24, 48, 168]
    for group in groups:
        for feature in features:
            for time in times:
                df[f'roll{time}_mean_{feature}'] = df.groupby(group)[feature] \
                    .transform(lambda x: x.rolling(window=time, min_periods=1).mean())
    return df

def ewm_features(df: pd.DataFrame):
    features = ['lag_Energia', 'lag_PrecEuro']
    groups = ['Codigo', 'Categoria']
    spans = [12, 24, 48, 168]
    for group in groups:
        for feature in features:
            for span in spans:
                df[f'ewm{span}_mean_{feature}'] = df.groupby(group)[feature] \
                    .transform(lambda x: x.ewm(span=span, min_periods=1).mean())
    return df

def diff_features(df: pd.DataFrame):
    features = ['lag_Energia', 'lag_PrecEuro']
    for feature in features:
        df[f'diff_{feature}'] = df.groupby('Codigo')[feature].diff()
        df[f'diff_{feature}'] = df.groupby(['Codigo'])[f'diff_{feature}'].transform(lambda x: x.fillna(x.mean()))
    return df

def volatility_features(df: pd.DataFrame):
    features = ['lag_Energia', 'lag_PrecEuro']
    groups = ['Codigo', 'Categoria']
    windows = [12, 24, 48, 168]
    for group in groups:
        for feature in features:
            for window in windows:
                df[f'volatility_{window}_{feature}'] = df.groupby(group)[feature] \
                    .transform(lambda x: x.rolling(window=window, min_periods=1).std())
                df[f'volatility_{window}_{feature}'] = df.groupby(['Codigo'])[f'volatility_{window}_{feature}'].transform(lambda x: x.fillna(x.mean()))

    return df

def fourrier_features(df: pd.DataFrame):    
    def apply_fft(group):
            X = fft.fft(group['lag_Energia'])
            N = len(X)
            group['lag_Energia_fft'] = np.abs(X) / N  # Normalize by length
            return group

    df = df.groupby('Codigo', group_keys=False).apply(apply_fft)
    return df

def frequency_power_features(df: pd.DataFrame):
    df['power_spectrum'] = df.groupby('Codigo')['lag_Energia'].transform(lambda x: np.abs(fft.fft(x))**2 / len(x))
    return df

### Feature Selection with RFE

In [ ]:
rfe_selector = RFE(estimator=xgb.XGBRegressor(), n_features_to_select=10, step=1)

## Pipeline

In [ ]:
pipeline = Pipeline([
    ('agg_features', FunctionTransformer(agg_features)),
    ('rolling_mean_features', FunctionTransformer(rolling_mean_features)),
    ('ewm_features', FunctionTransformer(ewm_features)),
    ('diff_features', FunctionTransformer(diff_features)),
    ('volatility_features', FunctionTransformer(volatility_features)),
    ('fourrier_features', FunctionTransformer(fourrier_features)),
    ('frequency_power_features', FunctionTransformer(frequency_power_features)),
    ('ordinal_encoding', OrdinalEncodingTransformer()),
    ('rfe', rfe_selector)
])

In [ ]:
def feature_transformation(x_train):
    x_train = agg_features(x_train)
    x_train = rolling_mean_features(x_train)
    x_train = ewm_features(x_train)
    x_train = diff_features(x_train)
    x_train = volatility_features(x_train)
    x_train = fourrier_features(x_train)
    x_train = frequency_power_features(x_train)
    x_train['zero_indicator'] = (x_train['lag_Energia'] == 0).astype(int)
    return x_train

## Models

### TimesFM

In [ ]:
class TimesFMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TimesFMModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [ ]:
def prepare_timesfm(X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.DataFrame) -> tuple:
    if isinstance(X_train, pd.DataFrame):
        X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
        X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
    elif isinstance(X_train, np.ndarray):
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    return X_train_tensor, y_train_tensor, X_test_tensor

In [ ]:
TIMESFM_MODEL_PARAMS = {
    'hidden_dim': 100,
    'output_dim': 1,
}

TIMESFM_TRAIN_PARAMS = {
    'lr': 0.01,
}


In [ ]:
def train_timesfm(
    X_train_tensor: torch.Tensor, 
    y_train_tensor: torch.Tensor,
    TIMESFM_MODEL_PARAMS: dict,
    TIMESFM_TRAIN_PARAMS: dict,
    epochs: int = 150
    ) -> TimesFMModel:

    timesfm_model = TimesFMModel(input_dim=X_train_tensor.shape[1], **TIMESFM_MODEL_PARAMS)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(timesfm_model.parameters(), **TIMESFM_TRAIN_PARAMS)
    
    timesfm_model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = timesfm_model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()
        
    return timesfm_model
    
def evaluate_timesfm(timesfm_model: TimesFMModel, X_test_tensor: torch.Tensor) -> np.ndarray:
    timesfm_model.eval()
    with torch.no_grad():
        y_pred_timesfm = timesfm_model(X_test_tensor).numpy().flatten()
        
    return y_pred_timesfm

### LSTM

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # batch_first=True makes input shape (batch, seq, feature)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Initialize hidden and cell states with zeros (and send to same device as x)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        
        # Use the output from the final time step
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [ ]:
def prepare_lstm(X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.DataFrame) -> tuple:
    selected_features = [
        'lag1_Energia', 'lag24_Energia', 'roll24_mean_lag_Energia', 'ewm24_mean_lag_Energia',
        'hour', 'day_of_week', 'day_of_month', 'month', 'is_weekend',
        'hour_sin', 'hour_cos', 'dow_sin', 'dow_cos',
        'PrecEuro', 'cum_energy'
    ]
    if isinstance(X_train, pd.DataFrame):
        X_train_selected = X_train[selected_features].values  
        X_test_selected  = X_test[selected_features].values
    elif isinstance(X_train, np.ndarray):
        X_train_selected = X_train[:, selected_features]
        X_test_selected  = X_test[:, selected_features]

    # Add a time dimension (sequence length = 1)
    X_train_lstm = torch.tensor(X_train_selected, dtype=torch.float32).unsqueeze(1)  
    X_test_lstm  = torch.tensor(X_test_selected, dtype=torch.float32).unsqueeze(1)
    y_train_lstm = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
    return X_train_lstm, y_train_lstm, X_test_lstm

In [ ]:
LSTM_MODEL_PARAMS = {
    'hidden_size': 50,
    'num_layers': 1,
    'output_size': 1
}

LSTM_TRAIN_PARAMS = {
    'lr': 0.01,
}


In [ ]:
def train_lstm(
    X_train: torch.Tensor, 
    y_train: torch.Tensor, 
    LSTM_MODEL_PARAMS: dict,
    LSTM_TRAIN_PARAMS: dict,
    epochs: int=150
    ) -> LSTMModel:
    
    input_size = X_train.shape[2]
    lstm_model = LSTMModel(input_size, **LSTM_MODEL_PARAMS)

    criterion_lstm = nn.MSELoss()
    optimizer_lstm = optim.Adam(lstm_model.parameters(), **LSTM_TRAIN_PARAMS)
    
    lstm_model.train()
    for epoch in range(epochs):
        optimizer_lstm.zero_grad()
        outputs = lstm_model(X_train)
        loss = criterion_lstm(outputs, y_train)
        loss.backward()
        optimizer_lstm.step()
        
    return lstm_model
    
    
def evaluate_lstm(lstm_model: LSTMModel, X_test_lstm: torch.Tensor) -> np.ndarray:
    lstm_model.eval()
    with torch.no_grad():
        y_pred_lstm = lstm_model(X_test_lstm).numpy().flatten()
    return y_pred_lstm

### XGB

In [ ]:
XGB_PARAMS = {
    'n_estimators': 1000,
    'early_stopping_rounds': 50,
    'objective': 'reg:squarederror',
    'max_depth': 3,
    'learning_rate': 0.01,
    'verbosity': 0,
    'booster': 'gbtree'
}

In [ ]:
def train_xgb(
    X_train: pd.DataFrame, 
    y_train: pd.Series, 
    X_test: pd.DataFrame,
    y_test: pd.Series,
    XGB_PARAMS: dict) -> xgb.XGBRegressor:    
    xgb_reg = xgb.XGBRegressor(**XGB_PARAMS)
    xgb_reg.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=False
    )
    return xgb_reg
    
def evaluate_xgb(xgb_reg: xgb.XGBRegressor, X_test: pd.DataFrame) -> np.ndarray:
    return xgb_reg.predict(X_test)
    

### LGBM

In [ ]:
LGBM_PARAMS = {
    'metric': 'mae',
    'learning_rate': 0.04139126441377782,
    'num_leaves': 59,
    'max_depth': 7,
    'min_data_in_leaf': 26,
    'feature_fraction': 0.9999701887398744,
    'bagging_fraction': 0.6553188710984839,
    'bagging_freq': 6,
    'lambda_l1': 0.1,
    'lambda_l2': 0.1,
    'verbose': -1,
    'seed': 42
}

In [ ]:
def train_lgbm(
    X_train: pd.DataFrame, 
    y_train: pd.Series, 
    X_test: pd.DataFrame,
    y_test: pd.Series,
    LGBM_PARAMS: dict
    ) -> lgb.LGBMRegressor:    
    lgbm_reg = lgb.LGBMRegressor(**LGBM_PARAMS)
    lgbm_reg.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)]
    )
    return lgbm_reg
    
def evaluate_lgbm(lgbm_reg: lgb.LGBMRegressor, X_test: pd.DataFrame) -> np.ndarray:
    return lgbm_reg.predict(X_test)

### Prophet

In [ ]:
def prepare_prophet(train: pd.DataFrame, TARGET: str) -> pd.DataFrame:
    train_prophet = train.reset_index()
    train_prophet['ds'] = pd.to_datetime(train_prophet['fechaHora'])

    train_prophet = train_prophet.rename(columns={TARGET: 'y'})

    return train_prophet[['ds', 'y', 'lag_PrecEuro', 'lag_Energia']]

def train_prophet(train_prophet: pd.DataFrame) -> Prophet:

    prophet_model = Prophet()
    prophet_model.add_regressor('lag_PrecEuro')
    prophet_model.add_regressor('lag_Energia')

    prophet_model.fit(train_prophet)
    
    return prophet_model

def evaluate_prophet(prophet_model: Prophet, test: pd.DataFrame) -> np.ndarray:

    test_prophet = test.reset_index()
    test_prophet['ds'] = pd.to_datetime(test_prophet['fechaHora'])
    future = test_prophet[['ds', 'lag_PrecEuro', 'lag_Energia']]

    # Forecast
    forecast = prophet_model.predict(future)
    return forecast['yhat'].values

## Benchmark

### Custom Rolling Window CV

In [ ]:
def custom_rolling_window_cv(data: pd.DataFrame, initial_train_window: int, forecast_horizon: int, step: int):
    """
    Custom rolling window cross-validation.
    """
    n = len(data)
    train_end = initial_train_window  
    while (train_end + forecast_horizon) <= n:
        train_idx = list(range(0, train_end))
        test_idx = list(range(train_end, train_end + forecast_horizon))
        yield train_idx, test_idx
        train_end += step

In [ ]:
nunique_codes = data['Codigo'].nunique()
INITIAL_TRAIN_WINDOW = 24*28*nunique_codes
FORECAST_HORIZON = 24*28*nunique_codes
STEP = 24*7*nunique_codes

data.drop('Descripcion', axis=1, inplace=True)

EXCLUDED_COLS = ['date', 'fechaHora', 'Energia', 'Energia_stationary', 'PrecEuro']
NUM_FEATURES = [col for col in data.select_dtypes(include=np.number).columns if col not in EXCLUDED_COLS]
CAT_FEATURES = [col for col in data.select_dtypes(exclude=np.number).columns if col not in EXCLUDED_COLS]
FEATURES = NUM_FEATURES + CAT_FEATURES

for col in CAT_FEATURES:
    if col in data.columns:
        data[col] = data[col].astype('category')

TARGET = 'Energia_stationary'

model_names = ['TimesFM', 'XGB', 'LGBM', 'Prophet']

results = {model: {'mae': [], 'mape': []} for model in model_names}

In [ ]:
data[FEATURES].info()

In [ ]:
def mae_score(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def mape_score(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / np.abs(y_true) + 1e-3))

In [ ]:
for fold, (train_idx, test_idx) in enumerate(custom_rolling_window_cv(data, INITIAL_TRAIN_WINDOW, FORECAST_HORIZON, STEP)):
    print(f"\n===== Fold {fold} =====")
    
    # Split the data into train and test folds
    train = data.iloc[train_idx]
    test = data.iloc[test_idx]
    
    # Common splits for models that require X and y inputs:
    X_train = train[FEATURES]
    y_train = train[TARGET]
    X_test = test[FEATURES]
    y_test = test[TARGET]
    
    # X_train = feature_transformation(X_train)
    # X_test = feature_transformation(X_test)
    
    # ordinal_encoder = OrdinalEncodingTransformer()
    # X_train = ordinal_encoder.fit_transform(X_train)
    # X_test = ordinal_encoder.transform(X_test)
        
    # X_train = pd.DataFrame(X_train, columns=X_train.columns.tolist())
    # X_test = pd.DataFrame(X_test, columns=X_test.columns.tolist())
    
    # --------------------------
    if 'Prophet' in model_names:
        data_prophet = prepare_prophet(train, TARGET)
        prophet_model = train_prophet(data_prophet)
        y_pred_prophet = evaluate_prophet(prophet_model, test)
        
        mae_prophet = mae_score(y_test, y_pred_prophet)
        mape_prophet = mape_score(y_test, y_pred_prophet)
        
        results['Prophet']['mae'].append(mae_score(y_test, y_pred_prophet))
        results['Prophet']['mape'].append(mape_prophet)
        
        print(f"Prophet    --> MAE: {mae_prophet:.4f}, MAPE: {mape_prophet:.4f}")
    
    # --------------------------
    if 'XGB' in model_names:
        xgb_model = train_xgb(X_train, y_train, X_test, y_test, XGB_PARAMS)
        y_pred_xgb = evaluate_xgb(xgb_model, X_test)
        
        mae_xgb = mae_score(y_test, y_pred_xgb)
        mape_xgb = mape_score(y_test, y_pred_xgb)
        
        results['XGB']['mae'].append(mae_xgb)
        results['XGB']['mape'].append(mape_xgb)
        
        print(f"XGBoost    --> MAE: {mae_xgb:.4f}, MAPE: {mape_xgb:.4f}")
    
    # --------------------------
    if 'LGBM' in model_names:
        lgb_train = lgb.Dataset(X_train, label=y_train, categorical_feature=CAT_FEATURES)
        lgb_val = lgb.Dataset(X_test, label=y_test, categorical_feature=CAT_FEATURES)
        
        # lgbm_model = train_lgbm(X_train, y_train,  X_test, y_test, LGBM_PARAMS)

        lgbm_model = lgb.train(
            LGBM_PARAMS,
            lgb_train,
            num_boost_round=1000,
            valid_sets=[lgb_train, lgb_val],
            callbacks=[lgb.early_stopping(stopping_rounds=50)]
        )
        

        y_pred_lgbm = evaluate_lgbm(lgbm_model, X_test)

        mae_lgbm = mae_score(y_test, y_pred_lgbm)
        mape_lgbm = mape_score(y_test, y_pred_lgbm)

        results['LGBM']['mae'].append(mae_lgbm)
        results['LGBM']['mape'].append(mape_lgbm)

        print(f"LightGBM   --> MAE: {mae_lgbm:.4f}, MAPE: {mape_lgbm:.4f}")

    # --------------------------
    if 'TimesFM' in model_names:
        X_train_timesfm, y_train_timesfm, X_test_timesfm = prepare_timesfm(X_train, y_train, X_test)          
        timesfm_model = train_timesfm(X_train_timesfm, y_train_timesfm, TIMESFM_MODEL_PARAMS, TIMESFM_TRAIN_PARAMS)
        y_pred_timesfm = evaluate_timesfm(timesfm_model, X_test_timesfm)
        mae_timesfm = mae_score(y_test, y_pred_timesfm)
        mape_timesfm = mape_score(y_test, y_pred_timesfm)
        
        results['TimesFM']['mae'].append(mae_timesfm)
        results['TimesFM']['mape'].append(mape_timesfm)
        
        print(f"TimesFM   --> MAE: {mae_timesfm:.4f}, MAPE: {mape_timesfm:.4f}")
    
    # --------------------------
    # LSTM Model (PyTorch)
    # --------------------------
    if 'LSTM' in model_names:
        X_train_lstm, y_train_lstm, X_test_timesfm = prepare_lstm(X_train, y_train, X_test)
        lstm_model = train_lstm(X_train_lstm, y_train_lstm, LSTM_MODEL_PARAMS, LSTM_TRAIN_PARAMS)
        y_pred_lstm = evaluate_lstm(lstm_model, X_test_timesfm)
        
        mae_lstm = mae_score(y_test, y_pred_lstm)
        mape_lstm = mape_score(y_test, y_pred_lstm)
        
        results['LSTM']['mae'].append(mae_lstm)
        results['LSTM']['mape'].append(mape_lstm)
        
        print(f"LSTM       --> MAE: {mae_lstm:.4f}, MAPE: {mape_lstm:.4f}")
    
# --------------------------
# STEP 5: Aggregate and Display Results
# --------------------------
benchmark_results = []
for model in model_names:
    avg_mae = np.mean(results[model]['mae'])
    avg_mape = np.mean(results[model]['mape'])
    benchmark_results.append({
        'Model': model,
        'MAE': avg_mae,
        'MAPE': avg_mape
    })

results_df = pd.DataFrame(benchmark_results)
print("\n===== Benchmark Results =====")
print(results_df)

### Feature Importances

In [ ]:
XGB_PARAMS = {
    'n_estimators': 1000,
    'early_stopping_rounds': 50,
    'objective': 'reg:squarederror',
    'max_depth': 3,
    'learning_rate': 0.01,
    'verbosity': 0,
    'booster': 'gbtree'
}

# Convert 'fechaHora' column to datetime
data['fechaHora'] = pd.to_datetime(data['fechaHora'])

# Split the data such that the test set represents the last 3 months
split_date = pd.to_datetime('2024-05-01')
train = data[data['fechaHora'] < split_date]
test = data[data['fechaHora'] >= split_date]

EXCLUDED_COLS = ['fechaHora', 'Energia', 'Energia_stationary']
FEATURES = [col for col in data.columns if col not in EXCLUDED_COLS]
TARGET = 'Energia_stationary'

# Common splits for models that require X and y inputs:
X_train = train[FEATURES]
y_train = train[TARGET]
X_test  = test[FEATURES]
y_test  = test[TARGET]

X_train = pipeline.fit_transform(X_train, y_train)
X_test = pipeline.transform(X_test)

xgb_model = train_xgb(X_train, y_train, X_test, y_test, XGB_PARAMS)
y_pred_xgb = evaluate_xgb(xgb_model, X_test)

mae_xgb = mae_score(y_test, y_pred_xgb)
mape_xgb = mape_score(y_test, y_pred_xgb)

print(f"XGBoost    --> MAE: {mae_xgb:.4f}, MAPE: {mape_xgb:.4f}")

In [ ]:
if isinstance(X_train, pd.DataFrame):
    feature_names = X_train.columns.tolist()
    # Create a DataFrame with feature names and their importance scores
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': xgb_model.feature_importances_
    })

    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    print(importance_df)